In [58]:
! pip install datasets transformers 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [60]:
from huggingface_hub import notebook_login
notebook_login()

In [61]:
!apt install git-lfs

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.


In [62]:
import transformers
import pandas as pd
print(transformers.__version__)

4.30.2


In [63]:
from transformers.utils import send_example_telemetry
send_example_telemetry("text_classification_notebook", framework="pytorch")

In [64]:
task = "rte"
model_checkpoint = "distilbert-base-uncased"
batch_size = 32

In [65]:
from datasets import load_metric
actual_task = "rte" if task == "rte" else task
metric = load_metric('glue', actual_task)

In [66]:
metric

Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(res

In [67]:
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'accuracy': 0.5625}

In [68]:
df = pd.read_csv("/kaggle/input/final-data/final_data_.csv")

In [69]:
df.head()

,rule,entry,label
0,"Authorized actions consist of ['Login', 'View'...",User ID is leswmroe Timestamp is 2023-01-02 04...,0
1,The tokens that users can employ are selected ...,User is AXBaCo TimeStamp is 2023-01-02 17:58:5...,0
2,"Belonging to the list [192.168.1.1, 192.168.1....",IP Address is 192.168.1.2 Timestamp is 2023-01...,1
3,It is permissible for users to possess tokens ...,User is Jack87 TimeStamp is 2023-01-01 22:56:2...,1
4,Users are allowed to utilize tokens with value...,User is Jack87 TimeStamp is 2023-01-03 18:15:0...,1


In [70]:
df['label'].value_counts()

1    65999
0    54001
Name: label, dtype: int64

In [71]:
from datasets import Dataset, DatasetDict
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['rule', 'entry', 'label'],
    num_rows: 120000
})

In [72]:
from sklearn.model_selection import train_test_split

train_testvalid = dataset.train_test_split(test_size=0.2)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

dataset

DatasetDict({
    train: Dataset({
        features: ['rule', 'entry', 'label'],
        num_rows: 96000
    })
    test: Dataset({
        features: ['rule', 'entry', 'label'],
        num_rows: 12000
    })
    valid: Dataset({
        features: ['rule', 'entry', 'label'],
        num_rows: 12000
    })
})

In [73]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [74]:
task_to_keys = {
    "rte": ("rule", "entry")
}
rule, entry = task_to_keys[task]

print(f"Rule: {dataset['train'][0][rule]}")
print(f"Entry: {dataset['train'][0][entry]}")

Rule: The tokens that users can employ are selected from [874264, 567290, 927634]. Acceptable usernames are represented by [Kris345, Jack87, Bella233].
Entry: User is hxYiPm TimeStamp is 2023-01-04 11:33:54 Token is 364144 Status Code is 200


In [75]:
def preprocess_function(param):
    return tokenizer(param[rule], param[entry], truncation=True)

In [76]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/96 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [77]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.we

In [78]:
metric_name = "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"compliance_monitoring_oms",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=4,
    weight_decay=0.001,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

In [79]:
args.callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]

In [80]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [81]:
!pip install wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [82]:
import wandb
wandb.init(project="compliance-monitoring", entity="vineetchotaliya30")

eval/accuracy,▁▁▁▁▁▁▁▁
eval/loss,█▃▄▁▁▁▁▁
eval/runtime,▄██▄▁▁▁▃
eval/samples_per_second,▅▁▁▅███▆
eval/steps_per_second,█▇▇▁▁▁▁▁
train/epoch,▁▃▆▁▃▆███
train/global_step,▂▅█▁▃▅▇▇▇
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁


In [83]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["valid"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/Vineetttt/compliance_monitoring_oms into local empty directory.


In [84]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.002200,0.002301,0.999667
2,0.001500,0.002159,0.999667
3,0.001400,0.001932,0.999667
4,0.001600,0.001992,0.999667


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=6000, training_loss=0.01011993279059728, metrics={'train_runtime': 5760.5981, 'train_samples_per_second': 66.66, 'train_steps_per_second': 1.042, 'total_flos': 2.956011656851123e+16, 'train_loss': 0.01011993279059728, 'epoch': 4.0})

In [85]:
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.002301492029801011,
 'eval_accuracy': 0.9996666666666667,
 'eval_runtime': 70.1316,
 'eval_samples_per_second': 171.107,
 'eval_steps_per_second': 2.681,
 'epoch': 4.0}

In [86]:
trainer.push_to_hub()

Upload file runs/Aug18_18-34-06_005627f08a7f/events.out.tfevents.1692383699.005627f08a7f.28.3:   0%|          …

Upload file runs/Aug18_18-34-06_005627f08a7f/events.out.tfevents.1692389530.005627f08a7f.28.4:   0%|          …

To https://huggingface.co/Vineetttt/compliance_monitoring_oms
   9aae60c..66ec3f8  main -> main

To https://huggingface.co/Vineetttt/compliance_monitoring_oms
   66ec3f8..ee42f73  main -> main



'https://huggingface.co/Vineetttt/compliance_monitoring_oms/commit/66ec3f8d22c1afe2adc3b43553b82faadb6f805a'

In [87]:
test_dataset = dataset["test"]

In [88]:
test_dataset

Dataset({
    features: ['rule', 'entry', 'label'],
    num_rows: 12000
})

In [89]:
dataset_test = test_dataset.map(preprocess_function, batched=True)

  0%|          | 0/12 [00:00<?, ?ba/s]

In [90]:
dataset_test

Dataset({
    features: ['rule', 'entry', 'label', 'input_ids', 'attention_mask'],
    num_rows: 12000
})

In [91]:
preds=trainer.predict(dataset_test)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [92]:
predictions = trainer.predict(dataset_test, metric_key_prefix="predict")

In [93]:
y_pred = np.argmax(predictions.predictions, axis=1)

In [94]:
import numpy as np
np.unique(y_pred)

array([0, 1])

In [95]:
pred_labels =pd.DataFrame(data=y_pred,columns=['label_pred'])
ground_truth_labels = dataset['test']['label']
ground_truth_df = pd.DataFrame({'ground_truth_label': ground_truth_labels})
comparison_df = pd.concat([ground_truth_df, pred_labels], axis=1)

In [113]:
comparison_df.head()

,ground_truth_label,label_pred,matching
0,0,0,True
1,1,1,True
2,0,0,True
3,0,0,True
4,1,1,True


In [115]:
print("Hello")

In [116]:
comparison_df.to_csv("output.csv",index=False)